**Set environment**

In [1]:
### set env
source ../config/config_duke.sh
source ../config/config_load_module_bedtools.sh
show_env

You are on Duke Server: HARDAC
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code
SING DIRECTORY (FD_SING): /data/reddylab/Kuei/singularity
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log



**Check data**

In [5]:
ls -1 ${FD_RES}/results/region/annotation_crispri_casa

crispri_casa_peak.bed


## Annotate ATAC peaks using CRISPRi FlowFISH CASA peaks

In [10]:
sbatch -p ${NODE} \
    --exclude=dl-01 \
    --cpus-per-task 8 \
    --mem 8G \
    --output ${FD_LOG}/annotation_astarr_peak_macs_input_crispri_casa.txt \
    --array 0 \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config/config_duke.sh

### set input and output
FD_BED=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_BED=KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed
FP_BED=${FD_BED}/${FN_BED}

FD_ANN=${FD_RES}/results/region/annotation_crispri_casa
FN_ANN=crispri_casa_peak.bed
FP_ANN=${FD_ANN}/${FN_ANN}

FD_OUT=${FD_RES}/results/region/KS91_K562_ASTARRseq_peak_macs_input
FN_OUT=peak.annotation.crispri_casa_peak.bed.gz
FP_OUT=${FD_OUT}/${FN_OUT}

### print start message
timer_start=`date +%s`
echo "Hostname:          " $(hostname)
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### show I/O file
echo "Input: " ${FP_BED}
echo
echo "show first few lines of input"
cat ${FP_BED} | head -5
echo
echo "Input: " ${FP_ANN}
echo
echo "show first few lines of input"
cat ${FP_ANN} | head -5
echo

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### annotation using intersect
bedtools intersect \
    -a ${FP_BED} \
    -b ${FP_ANN} \
    -wo \
    -f 0.5 \
    -F 0.5 \
    -e \
| gzip -c \
> ${FP_OUT}

### show output file
echo
echo "Output: " ${FP_OUT}
echo
echo "show first few lines of output:"
zcat ${FP_OUT} | head -5
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 29556841


**Check results**

In [11]:
cat ${FD_LOG}/annotation_astarr_peak_macs_input_crispri_casa.txt

Hostname:           x3-01-3.genome.duke.edu
Slurm Array Index:  0
Time Stamp:         03-07-23+16:25:01

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/KS91_K562_ASTARRseq_peak_macs_input/KS91_K562_hg38_ASTARRseq_Input.all_reps.masked.union_narrowPeak.q5.bed

show first few lines of input
chr1	10015	10442
chr1	14253	14645
chr1	16015	16477
chr1	17237	17772
chr1	28903	29613

Input:  /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/region/annotation_crispri_casa/crispri_casa_peak.bed

show first few lines of input
chr11	33944871	33945471	chr11:33944871-33945471	0.803764617	.	CAPRIN1	ENSG00000135387
chr11	34050971	34051471	chr11:34050971-34051471	0.973987294	.	CAPRIN1	ENSG00000135387
chr11	34051571	34054171	chr11:34051571-34054171	2.06429067	.	CAPRIN1	ENSG00000135387
chr11	34085871	34086871	chr11:34085871-34086871	1.009272431	.	CAPRIN1	ENSG00000135387
chr11	33944868	33945268	chr11:33944868-33945268	1.214675507	.	CAT	ENSG00000121691


Output:  /data/redd